Se carga el modelo y se obtienen los datos de la db que se utilizaran

In [1]:
import psycopg2
import numpy as np
from keras.models import load_model

sourceMarketId = 1
endMarketId = 40

x_db = []
x_min = 0
x_max = 0


try:
    connection = psycopg2.connect(user = "postgres", password = "r351d3nc14501", host = "127.0.0.1", port = "5432", database = "potatoe_markets")
    cursor = connection.cursor()
    
    postgreSQL_select_Query = "select price from normalized_price where sourcemarketid = {} and endmarketid = {} order by sniimdate desc limit 25".format(sourceMarketId, endMarketId)
    cursor.execute(postgreSQL_select_Query)
    #print("Selecting rows from mobile table using cursor.fetchall")
    value_records = cursor.fetchall()
    #Save each column value into the correspondent  array
    for row in value_records:
        x_db.append(row[0])
    
    postgreSQL_select_Query = "select (select marketName from market where marketid = {}), (select marketName from market where marketid = {})".format(sourceMarketId, endMarketId)
    cursor.execute(postgreSQL_select_Query)
    #print("Selecting rows from mobile table using cursor.fetchall")
    value_records = cursor.fetchall()
    #Save each column value into the correspondent  array
    print(value_records)
    sourceMarketName = value_records[0][0]
    endMarketName = value_records[0][1]
        
    postgreSQL_select_Query = "select min(price), max(price) from normalized_price"
    cursor.execute(postgreSQL_select_Query)
    #print("Selecting rows from mobile table using cursor.fetchall")
    value_records = cursor.fetchall()
    #Save each column value into the correspondent variable
    x_min = np.float32(value_records[0][0])
    x_max = np.float32(value_records[0][1])
    
except (Exception, psycopg2.Error) as error :
    print ("Error while fetching data from PostgreSQL", error)
finally:
    #closing database connection.
    if(connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")


modelName = "{} {}.h5".format(sourceMarketName, endMarketName).replace(':', '')
#model = load_model('Modelo_Zacatecas_Mercado_Abastos_Zacatecas.h5')
model = load_model(modelName)
        

def scaleValues(x, x_min, x_max, r_min = 0, r_max = 1):
    scale = (r_max - r_min) / (x_max - x_min)
    X_scaled = (scale * x) + r_min - (x_min * scale)
    return X_scaled


for con in range(0, len(x_db)):
    x_db[con] = scaleValues(float(x_db[con]), x_min, x_max)


x_db.reverse()
x_db = np.asarray(x_db)
x_db = x_db.reshape((1, x_db.shape[0]))


Using TensorFlow backend.
W1003 12:40:04.202665  6568 deprecation_wrapper.py:119] From c:\users\user\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1003 12:40:04.214663  6568 deprecation_wrapper.py:119] From c:\users\user\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1003 12:40:04.249567  6568 deprecation_wrapper.py:119] From c:\users\user\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W1003 12:40:04.250565  6568 deprecation_wrapper.py:119] From c:\users\user\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:181: The name tf.Co

[('Sinaloa', 'Zacatecas: Mercado de Abasto de Zacatecas')]
PostgreSQL connection is closed


Se realiza la prediccion de uno o varios elementos

In [2]:
#Funcion encargada para revertir la escala del valor predecido
def reverseScale(x_scaled, x_min, x_max, r_min = 0, r_max = 1):
    scale = (r_max - r_min) / (x_max - x_min)
    X = (x_scaled - r_min + (x_min * scale)) / scale  
    return X

#un elemento
p2 = model.predict(x_db.reshape(1,25), 10, verbose=0)
reverseScaledPrediction = reverseScale(p2[0][0], x_min, x_max)
print(reverseScaledPrediction)

13.744477987289429


In [3]:
weekdayPredictions = []
#5 elementos
for con in range(0, 5):
    pday = model.predict(x_db, 10, verbose=0)
    x_db = np.reshape(x_db, x_db.shape[1])
    x_db = np.delete(x_db, 0)
    x_db = np.append(x_db, pday)
    x_db = x_db.reshape(1,x_db.shape[0])
    reverseScaledPrediction = reverseScale(pday[0][0], x_min, x_max)
    weekdayPredictions.append(reverseScaledPrediction)

print(weekdayPredictions)

[13.744477987289429, 14.40894439816475, 15.066081404685974, 14.183828473091125, 13.668511837720871]
